# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [4]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6/" # SET
dataset_name = "force_sensor_trial6"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = " "  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-292409.xml.json"

zynq_data_csv = "/home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6/force_sensor_trial6.csv" 
rosbag_data_bag = "/home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6/force_sensor_trial6.bag"

unit_convert_path = dataset_path + "/" + dataset_name + "_unitConvert.csv"

# unit_convert_cmd = "python3 " + unit_convert_path + " -c " + si_unit_json_path + " -f " 


# ROSBAG PATHS
read_rosbag_script_py = "/home/ndrakes1/dvrk_force_estimation/preprocess/read_ros_bags.py"
preprocess_rosbag_script_py = "/home/ndrakes1/dvrk_force_estimation/preprocess/preprocess.py"


ORIGINAL_FREQ = 10000

INTERPOLATE = True


FILTER = True
FILTER_FREQ = 200
FILTER_VELOCITY = True

DOWNSAMPLE = False
DOWNSAMPLE_FREQ = 1000
DOWNSAMPLE_MOVING_AVERAGE = True

##################################################################################

# EMBEDDED

In [5]:
!. ~/catkin_ws/devel/setup.bash

## Unit Convert

In [6]:
!.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

Data written to /home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6/force_sensor_trial6_unitConvert.csv


## Extract Force Sensor Data

In [7]:
!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv

Saved SENSOR DATA CSV to /home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6//sensor/sensor.csv


## Extract Joint Data

In [8]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

## Filter Joint Data

### Preprocessed Training File Paths

In [9]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


### Interpolate

In [10]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    df = interpolate_dataframe_to_sample_rate(df, ORIGINAL_FREQ)



[0.00000000e+00 1.47048000e-04 2.61282000e-04 ... 1.38936641e+02
 1.38936641e+02 1.38936641e+02]


### FIR Filtering

In [11]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY)

### Downsample


In [12]:
from downsample import downsample_dataframe

if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### SAVE TO CSV

In [13]:
df.to_csv(data_file, index=False, header=False)


## Extract Jacobian

In [14]:
!mkdir {dataset_path}jacobian
!.venv/bin/python3 interpolate_jacobian2.py {unit_convert_path} {dataset_path}jacobian/interpolated_all_jacobian.csv dvpsm.rob

Flattened Jacobians (column-major) written to /home/ndrakes1/force_estimation_data_captures/capture_9-6-25/force_sensor_capture_6/jacobian/interpolated_all_jacobian.csv


# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE